In [1]:
! pip install -qU crewai groq langchain langchain-groq langchain-openai python-dotenv 'crewai[tools]'

In [11]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew, Process
from crewai_tools import WebsiteSearchTool, ScrapeWebsiteTool
from langchain.tools import DuckDuckGoSearchRun, Tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [10]:
#initializing LLM model
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.6,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [17]:
searchtool = DuckDuckGoSearchRun()

#wikipedia tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

def wikipedia_search(query):
    return wikipedia.run(query)

wikipedia_tool = Tool(
    name="WikipediaSearch",
    func=wikipedia_search,
    description="Search Wikipedia for information on a given topic. Input should be a search query."
)

web_scraper_tool = ScrapeWebsiteTool()

In [23]:
researcher = Agent(
    role='Researcher',
    goal='Conduct thorough research on the given query',
    backstory='You are an expert researcher with access to vast information',
    llm=llm,
    tools=[searchtool, wikipedia_tool, web_scraper_tool],
    verbose=True
        )

In [24]:
writer = Agent(
    role='Writer',
    goal='Synthesize research findings into a coherent report',
    backstory='You are a skilled writer capable of creating clear and concise reports',
    llm=llm,
    verbose=True
        )

In [41]:
fact_checker = Agent(
    role='Fact Checker',
    goal='Verify the accuracy of the information in the report and rewrite the article accordingly',
    backstory='You are a meticulous fact-checker with a keen eye for detail',
    tools=[searchtool],
    llm=llm,
    verbose = True
        )

In [42]:
def create_tasks(topic):
        research_task = Task(
            description=f"""Research the following query: {topic}. 
            Use the search tools to find relevant information, and the web scraper to gather details from specific websites.
            Use the DuckDuckGo search tool  and Wikipedia search tool by providing a clear and concise search query string.
            Use the WebScraper tool by providing a clear and valid website URL as a string.""",
            agent=researcher,
            expected_output="A comprehensive research summary on the given topic."
        )

        writing_task = Task(
            description="""Write a comprehensive report based on the research findings. 
            Organize the information logically and ensure clarity.""",
            agent=writer,
            expected_output="An engaging and informative article on the given topic."
        )

        fact_checking_task = Task(
            description="""Verify the accuracy of the information in the report. 
            Use search tools to cross-reference facts and highlight any discrepancies.
            Use the DuckDuckGo search tool by providing a clear and concise search query string.
            Once the information is verified and a report is created rephrase the article according to fact-check report 
            and provide the final article""",
            agent=fact_checker,
            expected_output="A detailed fact-checking report on the article. Along with the final article after doing fact-check"
        )

        return [research_task, writing_task, fact_checking_task]

In [43]:
def conduct_research(topic):
    tasks = create_tasks(topic)
    crew = Crew(
        agents=[researcher, writer, fact_checker],
        tasks=create_tasks(topic),
        verbose=True,
        task_execution_timeout=200
    )
    return crew.kickoff()

In [ ]:
if __name__ == "__main__":
    research_topic = input("Enter a research topic: ")
    result = conduct_research(research_topic)
    print(result)